In [ ]:
# import modules/libraries
import warnings 
warnings.simplefilter(action='ignore')
import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import time
from scipy import stats
import itertools
import os
import pickle
import geojson
from sqlalchemy import create_engine
import re
import sqlite3
from pathlib import Path
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import chardet
from scipy import spatial
from scipy.spatial import KDTree
from shapely import wkt

cwd = Path().resolve()

# visualisation
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib as mpl 
%matplotlib inline 
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

#### Data Engineering <a class="anchor" id="data-engineering"></a>

https://www.entsoe.eu/data/map/

https://transparency.entsog.eu/#/map



In [ ]:
df['price'] = df.apply(lambda x: get_price(x['price']), axis=1)

df['neighbourhood'] = df['neighbourhood'].str.replace('Landstra§e', 'Landstraße')
df['neighbourhood'] = df['neighbourhood'].str.replace('Rudolfsheim-Fnfhaus', 'Rudolfsheim-Fünfhaus')


In [ ]:
import json
def get_geo_data():
    """ load geojson data """
    with open(os.path.join(Path(cwd).parent, 'data', 'geojson', 'vienna.geojson'), encoding='utf-8') as fp:
        counties = geojson.load(fp)
    return counties

def save_figure(fig, name):
    with open(f"{name}.json", "w") as outfile:
        outfile.write(fig)
    

In [ ]:

def heatmap_airbnb2(title=''):
    """ """
    districts = get_geo_data()
    k = aggregate_data(df, 'neighbourhood', {'neighbourhood':['first'], 'price':['median'], 'host_is_superhost': ['first']},\
                       rename=['district', 'median', 'host_is_superhost'])
    k.sort_values(by='median', ascending=True, inplace=True)
    k['median'] = k['median'].astype('category')
    k.sort_values(by='median', ascending = False, inplace=True)
    fig = px.choropleth_mapbox(k, geojson=districts, locations=k['district'], featureidkey="properties.name", 
                               color=k['median'],
                               title=title,
                               color_discrete_sequence=px.colors.qualitative.Prism, 
                               labels={'median':'price per night'},
        mapbox_style="open-street-map", zoom=10, center = {"lat": 48.210033, "lon": 16.363449}, opacity=0.60)
    
    fig.add_scattermapbox(
        lat=df['latitude'].tolist(),
        lon=df['longitude'].tolist(),
        mode='markers',
        showlegend=False,
        #text=texts,
        marker_size=5,
        marker_color='#F3B5B6',
        opacity= 0.5,
        hoverinfo='skip'
    )
    fig.update_layout(font=dict(family="Helvetica"))
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(autosize=False,width=700,height=500)
    
    save_figure(fig.to_json(), "test")
    fig.show()

    
def heatmap_airbnb(title=''):
    """ """
    districts = get_geo_data()
    agg = df.groupby('neighbourhood').agg(nr_listings = ('id', 'count')).reset_index().sort_values('nr_listings', ascending=False)
    agg['ratio'] = 100 * agg['nr_listings'] / agg['nr_listings'].sum()
    agg['nr_listings'] = agg['nr_listings'].astype('category')
    agg.sort_values(by='nr_listings', ascending = False, inplace=True)
    fig = px.choropleth_mapbox(agg, geojson=districts, locations=agg['neighbourhood'], featureidkey="properties.name",
                               color_discrete_sequence=px.colors.qualitative.Dark24,
                               color=agg['nr_listings'],
                               #color=agg['ratio'],
                               title=title,
                               labels={'nr_listings':'Nr. of listings'},
        mapbox_style="open-street-map", zoom=10, center = {"lat": 48.210033, "lon": 16.363449}, opacity=0.40)
    
    fig.add_scattermapbox(
        lat=df['latitude'].tolist(),
        lon=df['longitude'].tolist(),
        mode='markers',
        #text=texts,
        marker_size=2,
        marker_color='#F3F5F6',
        opacity= 0.9,
        showlegend=True,
        hoverinfo='skip' #hoverinfo='none'
    )
    fig.update_layout(font=dict(family="Helvetica"))
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(autosize=False,width=700,height=500)
    
    save_figure(fig.to_json(), "test2")
    fig.show()


def bar_airbnb(df):
    """generates the bar chart of the category distribution from the "direct" genre """
    agg = df.groupby('neighbourhood').agg(nr_listings = ('id', 'count')).reset_index().sort_values('nr_listings', ascending=False)
    agg['ratio'] = 100 * agg['nr_listings'] / agg['nr_listings'].sum()
    fig = px.bar(x=agg['neighbourhood'].tolist(), y=agg['ratio'])
    save_figure(fig.to_json(), "barchart2")


In [ ]:
def num_reviews_over_time(data_reviews):
    fig = go.Figure()
    fig.add_trace(
    go.Scatter(x=data_reviews['date'], y=data_reviews['num_reviews'], showlegend=False, line=dict(color='#2c6e81', width=1)))
    fig.update_layout( font=dict(family="Open Sans"), legend_font_size=14),
    fig.add_trace(go.Scatter(x=data_reviews['date'], y=data_reviews['num_reviews'].rolling(20).mean(), 
                             name ="SMA 20", line=dict(color='orange', width=2), showlegend=True))
    # fig.update_layout(xaxis=dict(tickformat="%b"))
    fig.update_layout(yaxis_title="Nr. of reviews")
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    fig.update_layout(legend=dict(yanchor='top',y=.95,xanchor='left',x=0.01))
    fig.update_layout(autosize=False,width=700,height=350)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

df_reviews = df_rev.copy()
# Group by on date and count how many reviews were made on that day
data_reviews = df_reviews.groupby('date').count()['listing_id'].reset_index()
# Reset the column names
data_reviews.columns = ['date', 'num_reviews']
# Cast the datatype of the date column so that we can use plot_date.
data_reviews['date'] = pd.to_datetime(data_reviews['date'])
data_reviews.tail()

## Turn OpenStreetMap Location Data into ML Features <a class="anchor" id="osm-features"></a>
#### How to pull shops, restaurants, public transport modes and other local amenities into your ML models
With this information it might be possible to find a reasonable model with ammeniteis which affect what a host will charge for an airbnb.

##### Define Area of Interest
In this case the area of Vienna is imported from a geojson file into a geopandas dataframe. This datframe needs to be converted to a polygon.

In [ ]:
boundary_geojson = gpd.read_file(os.path.join(Path(cwd).parent, 'data', 'geojson', 'vienna.geojson'))
boundary_geojson.drop(columns=['cartodb_id', 'created_at', 'updated_at'], inplace=True)
region = boundary_geojson.geometry.unary_union

def get_local_crs(y,x):  
    x = ox.utils_geo.bbox_from_point((y, x), dist = 500, project_utm = True, return_crs = True)
    return x[-1]
  
# Set longitude and latitude of Vienna
lon_latitude = 48.210033
lon_longitude = 16.363449

local_utm_crs = get_local_crs(lon_latitude, lon_longitude)
# print(f"boundary data type: {type(boundary_geojson)}, region data type: {type(region)}")

##### Pull list of all amenities from OSM wiki page for reference
To get all the OSM queries especially for amenities, a list with all the keys is displayed from the [Openstreetmap wiki](https://wiki.openstreetmap.org/wiki/Key:amenity). This OSM data is accessible through the osmnx python module. 

In [ ]:
def get_all_amenities():
    """ get all amenitiy keys from OSM wiki """
    try:
        amenities = pd.read_html('https://wiki.openstreetmap.org/wiki/Key:amenity', skiprows = 0, header=0, attrs = {'class': 'wikitable sortable toptextcells zebra jquery-tablesorter'})[0]
       # amenities.drop(columns=['Element', 'Carto rendering','Photo', 'Unnamed: 6'], inplace=True, axis=1)
        amenities.drop(index=0, inplace=True)
        return amenities
    except Excepion as e:
        print("Amenities could not be found")


amenities = get_all_amenities()
print(f"First ten amenity keys: {', '.join(amenities['Value'].tolist()[0:10])}")

In [ ]:
def get_osm_data(region, data):
    df = ox.geometries.geometries_from_polygon(region, tags=data[0])
    df.to_csv(os.path.join(Path(cwd).parent, 'data', 'osm', 
                           f'{list(data[0].values())[0]}.csv'), columns=['geometry'])
    return df

t0 = time.time()
cafe = get_osm_data(region, [{'amenity':'cafe'}])


##### Query the OSM location data with osmnx

```python
def get_osm_data(region, data):
    df = ox.geometries.geometries_from_polygon(region, tags=data[0])
    df.to_csv(os.path.join(Path(cwd).parent, 'data', 'osm', 
                           f'{list(data[0].values())[0]}.csv'), columns=['geometry'])
    return df

t0 = time.time()
cafe = get_osm_data(region, [{'amenity':'cafe'}])
restaurant = get_osm_data(region, [{'amenity':'restaurant'}])
attraction = get_osm_data(region, [{'tourism':'attraction'}])
station = get_osm_data(region, [{'station':'subway'}])
bar = get_osm_data(region, [{'amenity':'bar'}])
biergarten = get_osm_data(region, [{'amenity':'biergarten'}])
fast_food = get_osm_data(region, [{'amenity':'fast_food'}])
pub = get_osm_data(region, [{'amenity':'pub'}])
nightclub = get_osm_data(region, [{'amenity':'nightclub'}])
theatre = get_osm_data(region, [{'amenity':'theatre'}])
university = get_osm_data(region, [{'amenity':'university'}])
attraction = get_osm_data(region, [{'tourism':'attraction'}])
shop = get_osm_data(region, [{'shop':'supermarket'}])

roads = ox.graph.graph_from_polygon(region)
forest = ox.geometries.geometries_from_polygon(region, tags = {'landuse': 'forest'})
rivers = ox.geometries.geometries_from_polygon(region, tags = {'waterway': 'river'})
#building: True means that every type of buildings will be downloaded
#buildings = ox.geometries.geometries_from_polygon(region, tags = {'building': True})
#kindergarten = ox.geometries.geometries_from_polygon(region, tags = {'amenity':'kindergarten'})
#secondary_roads = ox.geometries.geometries_from_polygon(region, tags = {'highway': 'secondary'})
print (f"Completed in {round(time.time() - t0)} s")
```

In [ ]:
def import_csv_to_gpd(name):
    """ import the csv file a gepandas dataframe """
    df = pd.read_csv(os.path.join(Path(cwd).parent, 'data', 'osm', f'{name}.csv'), sep=",")
    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, crs='epsg:4326')
    return gdf

restaurant = import_csv_to_gpd('restaurant')
cafe = import_csv_to_gpd('cafe')
attraction = import_csv_to_gpd('attraction')
subway = import_csv_to_gpd('subway')
bar = import_csv_to_gpd('bar')
biergarten = import_csv_to_gpd('biergarten')
fast_food = import_csv_to_gpd('fast_food')
pub = import_csv_to_gpd('pub')
nightclub = import_csv_to_gpd('nightclub')
theatre = import_csv_to_gpd('theatre')
university= import_csv_to_gpd('university')
shop = import_csv_to_gpd('supermarket')

##### Visualization of OSM Features:

```python
ax = boundary_geojson.plot(facecolor = '#494D4D', figsize=(85,85))
ax.set_facecolor('#2C2E2E')
#buildings['geometry'].plot(facecolor = '#C61313', edgecolor = '#C61313', linewidth = 3, markersize = 1, ax = ax)
forest.plot(facecolor = '#494D4D', edgecolor = '#ADC3B8', linewidth = 2, linestyle = ':', hatch ='x', ax=ax)
rivers.plot(edgecolor='#67A0C3', linewidth=6, linestyle='-', ax=ax)
ox.plot_graph(roads, edge_color='white', node_color='white', edge_linewidth=2, node_size=2, ax=ax)
ax.grid('on', which='major', axis='x', color = '#99A2A2')
ax.grid('on', which='major', axis='y', color = '#99A2A2')
plt.show()
```

#### Turning the Data Into Features
What we currently have is a dataframe of all the restaurants in London. For a machine learning model, what we need is the number of restaurants within a 10-minute walk each Airbnb property. Quite a bit of manipulation is required to get to this.

The geo dataframe has a column defining the geometry (e.g. POINT, POLYGON, Mult-Polygon). Restaurant locations for instance come as a point object and the latitude and longitude coordination need to be extracted.

##### Convert Polygons into Points
Most properties are returned as a single point coordinate. However, some are returned in a different shape e.g POLYGON, MulitPOLYGON or LINESTRING (e.g. on larger properties). In order to work with polygons, only their center point is used for subsequent feature extraction and modelling.

In [ ]:
def get_lat_long(point):
    """ get latitude and longitude coordinate from POINT geometry """
    try:
        return pd.Series([point.x, point.y])
    except:
        pass

def geo_coordinates(df):
    """ import from csv in geopandas dataframe
    source: https://stackoverflow.com/questions/61122875/geopandas-how-to-read-a-csv-and-convert-to-a-geopandas-dataframe-with-polygons
    """
    df['geometry'] = df['geometry'].apply(lambda x: x.centroid if type(x) == Polygon else (x.centroid if type(x) == MultiPolygon else x))
    df[['long', 'lat']] = df.apply(lambda x: get_lat_long(x['geometry']), axis=1)
    df = df[df['geometry'].apply(lambda x : x.type=='Point' )]
    df = df.to_crs(local_utm_crs)
    return df

In [ ]:
restaurant = geo_coordinates(restaurant)
cafe = geo_coordinates(cafe)
bar = geo_coordinates(bar)
subway = geo_coordinates(subway)
biergarten = geo_coordinates(biergarten)
fast_food = geo_coordinates(fast_food)
pub = geo_coordinates(pub)
nightclub = geo_coordinates(nightclub)
theatre = geo_coordinates(theatre)
university = geo_coordinates(university)
attraction = geo_coordinates(attraction)
shop = geo_coordinates(shop)

##### Calculate Distances with a KD Tree
Iterate through each AirBnb property and work out how many respective Openstreetmap features there are within a radius of 1 km. This is done using a KD Tree which is an efficient way of searching through our 12,000 AirBnbs rooms and thousend of features figuring out which ones are close.

In [ ]:
def get_tree(df):
    try:
        # turn long/lats into a list
        coords = list(zip(df.geometry.apply(lambda x: x.y).values,df.geometry.apply(lambda x: x.x).values))
        # create a KDTree
        tree = spatial.KDTree(coords)
        return tree
    except Exception as e:
        print(e)

In a next step a function is created which is performed on each of the Airbnb properties. The function will query the tree and find the 500 closest restaurants along with calculating their distances from the Airbnb property. We use a figure of 500 in the hope that no property has more than 500 restaurants close to it.
With this approach it can be determined how many restaurants, bars, shops, subway stations, tourist hotspots, public parks etc. there are within a 10-minute walk of each Airbnb property.

In [ ]:
def find_points_closeby(tree, lat_lon, k = 500, max_distance = 500 ):
    results = tree.query((lat_lon), k = k, distance_upper_bound= max_distance)
    zipped_results = list(zip(results[0], results[1]))
    zipped_results = [i for i in zipped_results if i[0] != np.inf]
    return len(zipped_results)

t0 = time.time()
air_gdf = df.copy()

parameters = [restaurant, cafe , bar, subway, biergarten, fast_food, pub, nightclub,theatre,university,attraction]
names = ['restaurant', 'cafe', 'bar', 'subway', 'biergarten', 'fast_food', 'pub', 'nightclub','theatre','university','attraction']

air_gdf = gpd.GeoDataFrame(air_gdf, geometry = gpd.points_from_xy(air_gdf.longitude, air_gdf.latitude), crs = 4326)
air_gdf = air_gdf.to_crs(local_utm_crs)

for name, i in zip(names, parameters):
    tree = get_tree(i)
    air_gdf[name] = air_gdf.apply(lambda row: find_points_closeby(tree, (row.geometry.y, row.geometry.x)) , axis = 1)

print (f"Completed in {round(time.time() - t0)} s")

##### Correlation Heatmap

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(air_gdf.corr(), ax=ax, annot=True, fmt=".2f");
# fig.savefig("correlation-matrix.png") 

##### Visualization

In [ ]:
def add_amenities(fig, df, showlegend=True, name='', marker_color='rgb(135, 60, 200)', marker_size=5, marker='circle'):
    fig.add_scattermapbox(
        lat=df.lat.tolist(),
        lon=df.long.tolist(),
        #marker_symbol=marker,
        mode='markers', #'markers+text'
        #text=texts,
        #marker = marker,
        marker_size=marker_size,
        marker_color=marker_color,
        opacity= 0.8,
        showlegend=showlegend,
                #hover_data=['amenity'],
       # hoverinfo='restaurant'
      #  label={'trace 1':'test'},
        name=name)

def heatmap(df, title=''):
    """ selector specifies the geographic resolution 
    source: https://stackoverflow.com/questions/67680264/combining-mapbox-choropleth-with-additional-layers-and-markers-in-python-try-to
    additional markers : https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter_mapbox.html
    https://plotly.com/python/scatter-plots-on-maps/
    """
    feat_key = ''
    locations = ''
    hover_data = ''
    j = get_geo_data()
    fig = px.choropleth_mapbox(df, geojson=j, locations=test['name'], featureidkey="properties.name", color=test['value'],
                               title=title,
        mapbox_style="open-street-map", zoom=10, center = {"lat": 48.210033, "lon": 16.363449}, opacity=0.15)
    
    lons = [item['geometry']['coordinates'][0][0][0][1] for item in j['features']]
    lats = [item['geometry']['coordinates'][0][0][0][0] for item in j['features']]
    texts = [item['properties']['name'] for item in j['features']]

    marker = itertools.cycle(('circle')) 
    add_amenities(fig, fast_food, name='Restaurants', marker_color='rgb(240, 240, 200)', marker_size=12, marker=next(marker))
    add_amenities(fig, pub, name='Pubs', marker_color='rgb(255, 87, 200)', marker_size=1, marker=next(marker)) 
   
    add_amenities(fig, bar, name='Bar', marker_color='rgb(135, 60, 200)', marker_size=5, marker=next(marker))
    add_amenities(fig, cafe, name='Cafe', marker_color='rgb(135, 60, 200)', marker_size=7, marker=next(marker))    
    add_amenities(fig, fast_food, name='Fast food', marker_color='rgb(135, 60, 200)', marker_size=8, marker=next(marker)) 
    add_amenities(fig, subway, name='Subway', marker_color='rgb(135, 60, 200)', marker_size=6, marker=next(marker)) 
    add_amenities(fig, biergarten, name='Biergarten', marker_color='rgb(135, 60, 200)', marker_size=5, marker=next(marker)) 
    add_amenities(fig, attraction, name='Attraction', marker_color='rgb(135, 60, 200)', marker_size=5, marker=next(marker)) 
    add_amenities(fig, university, name='University', marker_color='rgb(40, 120, 43)', marker_size=2, marker=next(marker)) 
    add_amenities(fig, nightclub, name='Nightclub', marker_color='rgb(45, 67, 12)', marker_size=5, marker=next(marker))

    fig.update_coloraxes(showscale=False)
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(autosize=False,width=700,height=500)
    fig.show()

test = boundary_geojson.copy()

test.reset_index(inplace=True)
test['value'] =  test['index']*1.2
heatmap(boundary_geojson)

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, make_scorer

def display_results(cv, y_test, y_pred):
    """ check how well the model performs. """
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

def evaluate_model(model, X_test, y_test, category_names):
    """ evaluate how well the given model performs with test data set """
    y_pred = model.predict(X_test)

    class_report = classification_report(y_test, y_pred, target_names=category_names)
    print(class_report)

def save_model(model, model_filepath):
    """ save model as a .pkl file under a give file path """
    with open(model_filepath, 'wb') as file:
        pickle.dump(model, file)


# Separate the target variable and rest of the variables
X, y = air_gdf[['restaurant','cafe', 'bar', 'subway','biergarten','fast_food','pub','nightclub',
                'theatre','university']], air_gdf['price']

# Convert the dataset into an optimized data structure called Dmatrix that XGBoost supports and gives it acclaimed performance and efficiency gains. You will use this later in the tutorial.
data_dmatrix = xgb.DMatrix(data=X,label=y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123)
model = xgb.XGBRegressor()

model.fit(X_train,y_train)

save_model(model, os.path.join(Path(cwd).parent, 'model', 'xboost.pkl'))

# feature importance
print(model.feature_importances_)
preds = model.predict(X_test)

# Compute the rmse by invoking the mean_sqaured_error function from sklearn's metrics module.
rmse = np.sqrt(mean_squared_error(y_test, preds))

params = {}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

print("RMSE: %f" % (rmse))
print((cv_results["test-rmse-mean"]).tail(1))

xgb.plot_importance(model)
plt.rcParams['figure.figsize'] = [6, 6]
plt.show()

In [ ]:
# example to predict benchmark price
d = {'restaurant': 3, 'cafe': 100, 'bar':5, 'subway':3, 'biergarten':1, 'fast_food':15, 'pub':3,
    'nightclub':1,'theatre':0,'university':0}
X_pred = pd.DataFrame(data=d, index=[0])
preds = model.predict(X_pred)
print(preds)

### Results

In my final XGBoost model, as you can see below, these OSM features (highlighted in red) ended up being some of the most important drivers of price in London.


#### Convert notebook to html
```python
!jupyter nbconvert --to html Airbnb-Analysis.ipynb
```

In [ ]:
# download notebook to html
!jupyter nbconvert --to html Airbnb-Analysis.ipynb

In [ ]:
df.to_csv(os.path.join(Path(cwd).parent, 'data', f'airbnb_dataframe.csv'), columns=['id', 'price', 'latitude', 'longitude', 'neighbourhood'], sep=',', index=False, encoding='utf-8')